# Pseudobulk differential expression to compare pGL2-BRI1-GFP/bri1-T vs bri1-T

In [1]:
library(tidyverse)
library(Seurat)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(GeneOverlap)
library(gprofiler2)
library(ggrepel)
library(muscat)
library(purrr)
library(limma)
library(scran)
library(future)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Attaching SeuratObject

Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex hea

In [2]:
#for 200gb ram 
options(future.globals.maxSize = 200000 * 1024^2)

In [3]:
rc.integrated <- readRDS("../data/integrations/rc.integrated_15S_BRZ_bri1_Li_seu3_clean.rds")

In [4]:
rc.integrated

An object of class Seurat 
71252 features across 78435 samples within 3 assays 
Active assay: integrated (17338 features, 17338 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

## Cell type and developmental stage metadata

- Developmental stage: `time_zone`
- Cell type:`cell_type`
- Combination of cell type and developmental stage: `time_zone_cell_type`
- Combination of cell type and developmental stage with cell subtypes (not used): `time_zone_cell_subtypes` 

In [5]:
feature_names <- read_tsv("./data/features.tsv.gz", col_names = c("AGI", "Name", "Type")) %>%
  select(-Type) %>%
  distinct()


── Column specification ────────────────────────────────────────────────────────
cols(
  AGI = col_character(),
  Name = col_character(),
  Type = col_character()
)




# Differential State Analysis with Muscat

In [6]:
# subset samples you want to compare

integrated.de <- subset(rc.integrated, subset = sample %in% c("sc_133", "sc_137","sc_131","sc_135"))


In [7]:
integrated.de

An object of class Seurat 
71252 features across 20527 samples within 3 assays 
Active assay: integrated (17338 features, 17338 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

In [8]:
# code control vs treated manually

integrated.de$treated <- integrated.de$genotype %in% c("pGL2_BRI1_GFP_bri1_T")


integrated.de$treatment <- as.character(integrated.de$treated)

integrated.de$treatment<- gsub("TRUE", "pGL2_BRI1_bri1_T", integrated.de$treatment, ignore.case = FALSE, perl = FALSE,
     fixed = T, useBytes = FALSE)
  
integrated.de$treatment<- gsub("FALSE", "bri1_T", integrated.de$treatment, ignore.case = FALSE, perl = FALSE,
     fixed = T, useBytes = FALSE)

integrated.de$treatment <- factor(integrated.de$treatment, levels=c("bri1_T", "pGL2_BRI1_bri1_T"))


table(integrated.de$treatment)


          bri1_T pGL2_BRI1_bri1_T 
           12649             7878 

## Convert to sce

In [9]:
#  construct sce manually
my_metadata <- data.frame(sample_id = integrated.de$sample,
                              group_id = integrated.de$treatment,
                              cluster_id = integrated.de$time_zone_cell_type, 
                             date=integrated.de$rep) # include experimental rep as co-variate

sce <- SingleCellExperiment(assays = list(counts = integrated.de@assays$RNA@counts),
	                            colData = my_metadata)

In [10]:
    (sce <- prepSCE(sce, 
        kid = "cluster_id", # subpopulation assignments
        gid = "group_id",   # group IDs (ctrl/stim)
        sid = "sample_id",    # sample IDs (ctrl/stim.1234)
        drop = FALSE))        # drop all other colData columns

class: SingleCellExperiment 
dim: 29347 20527 
metadata(1): experiment_info
assays(1): counts
rownames(29347): AT1G01010 AT1G01020 ... AT5G53640 AT5G60260
rowData names(0):
colnames(20527): AAACCCAAGCATCCCG_9 AAACCCAAGGTTGAGC_9 ...
  TTTGGTTGTGTTATCG_15 TTTGGTTTCTGGGCCA_15
colData names(4): cluster_id sample_id group_id date
reducedDimNames(0):
altExpNames(0):

## pre-filtering

In [11]:
# remove undetected genes
sce <- sce[rowSums(counts(sce) > 0) > 0, ]
dim(sce)

[1] 26830 20527

In [12]:
# remove lowly expressed genes
sce <- sce[rowSums(counts(sce) > 1) >= 1, ]
dim(sce)

[1] 23075 20527

In [13]:
# create pseudobulk profiles
pb <- aggregateData(sce,
    assay = "counts", fun = "sum",
    by = c("cluster_id", "sample_id"))
# one sheet per subpopulation
assayNames(pb)

[1] "Distal Columella"                  "Distal Lateral Root Cap"          
 [3] "Elongation_Atrichoblast"           "Elongation_Cortex"                
 [5] "Elongation_Endodermis"             "Elongation_Pericycle"             
 [7] "Elongation_Phloem"                 "Elongation_Procambium"            
 [9] "Elongation_Trichoblast"            "Elongation_Xylem"                 
[11] "Maturation_Atrichoblast"           "Maturation_Cortex"                
[13] "Maturation_Endodermis"             "Maturation_Pericycle"             
[15] "Maturation_Phloem"                 "Maturation_Procambium"            
[17] "Maturation_Trichoblast"            "Maturation_Xylem"                 
[19] "Proliferation Domain_Atrichoblast" "Proliferation Domain_Cortex"      
[21] "Proliferation Domain_Endodermis"   "Proliferation Domain_Pericycle"   
[23] "Proliferation Domain_Phloem"       "Proliferation Domain_Procambium"  
[25] "Proliferation Domain_Trichoblast"  "Proliferation Domain_Xylem"       
[27] "Proximal Columella"                "Proximal Lateral Root Cap"        
[29] "Transition Domain_Atrichoblast"    "Transition Domain_Cortex"         
[31] "Transition Domain_Endodermis"      "Transition Domain_Pericycle"      
[33] "Transition Domain_Phloem"          "Transition Domain_Procambium"     
[35] "Transition Domain_Trichoblast"     "Transition Domain_Xylem"

In [14]:
# pseudobulks for 1st subpopulation
t(head(assay(pb)))

,AT1G01010,AT1G01020,AT1G01030,AT1G01040,AT1G01050,AT1G01060
sc_131,42,78,8,16,983,3
sc_133,67,97,21,17,1325,3
sc_135,9,42,3,9,521,1
sc_137,36,59,33,6,511,8


In [15]:
bscs <- read.csv("./data/GEO_upload_BR_time_scRNA_samples_metadata_with_stats.csv", na.strings=c("","NA"), stringsAsFactors = F)
bscs$date <- gsub('^([0-9]{4})([0-9]{2})([0-9]+)$', '\\1-\\2-\\3', bscs$date)

bscs

sample,name,genotype,full_genotype,treatment,concentration,age,timepoint,time_trt,rep,date,seq_run,X10X_chemistry,number.of.cells,number.of.genes.detected,Median.UMI.counts.per.cell,Median.number.of.genes.detected.per.cell
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
sc_1,WT control,WT,WT Col-0,Control,DMSO,7_day,NA,Control,1,2019-12-14,Nolan_6131,v3,9759,21844,3028.0,1528.0
sc_2,WT BRZ control,WT,WT Col-0,BRZ,1uM,7_day,0.0,BRZ,1,2019-12-14,Nolan_6131,v3,7512,22403,6274.0,2497.0
sc_5,WT BRZ then 2 hour BL,WT,WT Col-0,BL,100nM,7_day,2.0,2_hour_BL,1,2019-12-14,Nolan_6131,v3,9790,22656,5415.5,2421.5
sc_43,WT BRZ control,WT,WT Col-0,BRZ,1uM,7_day,0.0,BRZ,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,8874,22534,4936.0,2071.0
sc_44,WT BRZ then 30 mins BL,WT,WT Col-0,BL,100nM,7_day,0.5,0.5_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6843,22046,6935.0,2441.0
sc_45,WT BRZ then 1 hour BL,WT,WT Col-0,BL,100nM,7_day,1.0,1_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6336,22434,5525.5,2067.0
sc_46,WT BRZ then 2 hour BL,WT,WT Col-0,BL,100nM,7_day,2.0,2_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6847,22490,7506.0,2632.0
sc_47,WT BRZ then 4 hour BL,WT,WT Col-0,BL,100nM,7_day,4.0,4_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6907,22893,6430.0,2297.0
sc_48,WT BRZ then 8 hour BL,WT,WT Col-0,BL,100nM,7_day,8.0,8_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6389,22873,5230.0,2014.0


In [16]:
# experiment info for contrasts, add dates from csv

ei <- metadata(sce)$experiment_info
ei

sample_date <- select(bscs, sample_id=sample, date=rep)

ei <- left_join(ei, sample_date)

ei

sample_id,group_id,n_cells
<fct>,<fct>,<dbl>
sc_131,bri1_T,7621
sc_133,pGL2_BRI1_bri1_T,5975
sc_135,bri1_T,5028
sc_137,pGL2_BRI1_bri1_T,1903


Joining, by = "sample_id"



sample_id,group_id,n_cells,date
<chr>,<fct>,<dbl>,<int>
sc_131,bri1_T,7621,1
sc_133,pGL2_BRI1_bri1_T,5975,1
sc_135,bri1_T,5028,2
sc_137,pGL2_BRI1_bri1_T,1903,2


In [17]:
mm <- model.matrix(~ 0 + ei$group_id + ei$date)
dimnames(mm) <- list(ei$sample_id, c(levels(ei$group_id), "date"))

mm

,bri1_T,pGL2_BRI1_bri1_T,date
sc_131,1,0,1
sc_133,0,1,1
sc_135,1,0,2
sc_137,0,1,2


In [18]:
contrast <- makeContrasts("pGL2_BRI1_bri1_T-bri1_T", levels = mm)

contrast

,pGL2_BRI1_bri1_T-bri1_T
bri1_T,-1
pGL2_BRI1_bri1_T,1
date,0


In [19]:
res <- pbDS(pb, design = mm, 
            contrast = contrast, 
            method="edgeR", 
            min_cells=5, 
            filter = c("none"))

Distal Columella..Distal Lateral Root Cap..Elongation_Atrichoblast..Elongation_Cortex..Elongation_Endodermis..Elongation_Pericycle..Elongation_Phloem..Elongation_Procambium..Elongation_Trichoblast..Elongation_Xylem..Maturation_Atrichoblast..Maturation_Cortex..Maturation_Endodermis..Maturation_Pericycle..Maturation_Phloem..Maturation_Procambium..Maturation_Trichoblast..Maturation_Xylem..Proliferation Domain_Atrichoblast..Proliferation Domain_Cortex..Proliferation Domain_Endodermis..Proliferation Domain_Pericycle..Proliferation Domain_Phloem..Proliferation Domain_Procambium..Proliferation Domain_Trichoblast..Proliferation Domain_Xylem..Proximal Columella..Proximal Lateral Root Cap..Transition Domain_Atrichoblast..Transition Domain_Cortex..Transition Domain_Endodermis..Transition Domain_Pericycle..Transition Domain_Phloem..Transition Domain_Procambium..Transition Domain_Trichoblast..Transition Domain_Xylem..

### DEG results

In [20]:
# DEG results with gene freqs
(res_to_write_frq <- resDS(sce, res, bind = "row", cpm=TRUE, frq=T))

gene,cluster_id,sc_131.cpm,sc_135.cpm,sc_133.cpm,sc_137.cpm,sc_131.frq,sc_135.frq,sc_133.frq,sc_137.frq,bri1_T.frq,pGL2_BRI1_bri1_T.frq,logFC,logCPM,F,p_val,p_adj.loc,p_adj.glb,contrast
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AT1G01010,Distal Columella,8.480,3.610,10.500,10.500,0.06470,0.03400,0.07790,0.12900,0.05560,0.08910,0.7640,3.1800,3.82000,5.06e-02,0.44700,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01020,Distal Columella,15.800,16.800,15.200,17.300,0.10900,0.14700,0.09730,0.18900,0.12000,0.11800,-0.0114,4.0600,0.00111,9.73e-01,0.99400,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01030,Distal Columella,1.620,1.200,3.290,9.650,0.01260,0.01130,0.02430,0.10300,0.01220,0.04170,1.8600,2.1200,11.60000,6.66e-04,0.02500,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01040,Distal Columella,3.230,3.610,2.670,1.750,0.02210,0.03400,0.02070,0.02150,0.02560,0.02090,-0.5610,1.7000,1.00000,3.17e-01,0.90400,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01050,Distal Columella,199.000,209.000,208.000,149.000,0.69200,0.82300,0.71400,0.73000,0.73100,0.71800,-0.2060,7.5800,0.49600,4.81e-01,0.92400,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01060,Distal Columella,0.606,0.401,0.470,2.340,0.00473,0.00377,0.00365,0.02150,0.00445,0.00758,0.9680,0.4390,0.68700,4.07e-01,0.92400,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01070,Distal Columella,1.820,0.401,0.314,0.000,0.01420,0.00377,0.00243,0.00000,0.01110,0.00190,-2.5600,0.2070,3.00000,8.30e-02,0.56800,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01080,Distal Columella,0.404,0.000,0.941,0.000,0.00315,0.00000,0.00487,0.00000,0.00222,0.00379,1.1300,-0.1360,0.37000,5.43e-01,0.92400,1,pGL2_BRI1_bri1_T-bri1_T
AT1G01090,Distal Columella,41.400,37.700,36.800,39.500,0.27000,0.30200,0.24100,0.33500,0.27900,0.26200,-0.0564,5.3000,0.03160,8.59e-01,0.98400,1,pGL2_BRI1_bri1_T-bri1_T


In [21]:
## all genes background 

all_bg <- filter(res_to_write_frq,
                bri1_T.frq >=0.1 | pGL2_BRI1_bri1_T.frq >=0.1)

In [22]:
length(unique(all_bg$gene))

[1] 18565

In [23]:
#total DE genes p_adj.loc < 0.05, abs(logFC) > 1.5
sig_DE <- filter(res_to_write_frq, p_adj.loc<=0.05 & abs(logFC) > log2(1.5))
sig_DE <- left_join(sig_DE, feature_names, by=c("gene"="AGI"))

length(unique(sig_DE$gene))

[1] 9277

In [24]:
# filter gene freqs to avoid calling lowly detected genes
sig_DE_fil <- filter(sig_DE, bri1_T.frq >=0.1 | pGL2_BRI1_bri1_T.frq >=0.1)

In [25]:
length(unique(sig_DE_fil$gene))

[1] 8046

In [26]:
# load TFs
TF_list <- read_csv("./data/Kay_TF_thalemine_annotations.csv", col_names = c("gene", "TF_Name", "Description")) 


── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  TF_Name = col_character(),
  Description = col_character()
)




In [27]:
sig_DE_fil <- left_join(sig_DE_fil, TF_list)

Joining, by = "gene"



In [28]:
# label up vs down
sig_DE_fil <- sig_DE_fil %>%
  mutate(up_dn_label = case_when(logFC >=log2(1.5) ~ "Up",  
                                       logFC <=log2(1/1.5) ~ "Down",
                                       TRUE ~ "Not DE"))

sig_DE_fil$clust_up_dn <- paste(sig_DE_fil$cluster_id, sig_DE_fil$up_dn_label, sep="_")

sig_DE_fil

gene,cluster_id,sc_131.cpm,sc_135.cpm,sc_133.cpm,sc_137.cpm,sc_131.frq,sc_135.frq,sc_133.frq,sc_137.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01470,Distal Columella,3270.00,3330.00,5940.00,5830.00,0.9860,0.9960,0.9890,0.9960,⋯,14.6,1.32e-04,7.12e-03,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,LEA14,NA,NA,Up,Distal Columella_Up
AT1G01480,Distal Columella,8.08,5.21,11.40,22.20,0.0599,0.0491,0.0852,0.2790,⋯,10.0,1.56e-03,4.62e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,ACS2,NA,NA,Up,Distal Columella_Up
AT1G01725,Distal Columella,4.44,4.01,26.70,26.90,0.0300,0.0377,0.1850,0.2880,⋯,45.8,1.34e-11,5.70e-09,7.77e-06,pGL2_BRI1_bri1_T-bri1_T,AT1G01725,NA,NA,Up,Distal Columella_Up
AT1G02390,Distal Columella,15.80,19.20,46.70,28.70,0.0978,0.1430,0.2310,0.2400,⋯,10.8,9.94e-04,3.35e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,GPAT2,NA,NA,Up,Distal Columella_Up
AT1G02860,Distal Columella,56.40,46.10,135.00,175.00,0.3010,0.2830,0.5070,0.7380,⋯,25.8,3.80e-07,5.58e-05,2.19e-01,pGL2_BRI1_bri1_T-bri1_T,BAH1,NA,NA,Up,Distal Columella_Up
AT1G03200,Distal Columella,4.24,6.02,11.30,62.00,0.0284,0.0491,0.0730,0.3990,⋯,38.0,7.10e-10,1.95e-07,4.11e-04,pGL2_BRI1_bri1_T-bri1_T,AT1G03200,NA,NA,Up,Distal Columella_Up
AT1G04040,Distal Columella,55.80,69.80,36.70,23.70,0.3440,0.4570,0.2310,0.2400,⋯,10.8,9.99e-04,3.35e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,AT1G04040,NA,NA,Down,Distal Columella_Down
AT1G05620,Distal Columella,54.50,51.70,31.50,20.80,0.3140,0.3360,0.2020,0.2270,⋯,10.3,1.30e-03,4.09e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,URH2,NA,NA,Down,Distal Columella_Down
AT1G06080,Distal Columella,2.42,1.20,22.90,10.20,0.0174,0.0113,0.1160,0.1160,⋯,47.5,5.70e-12,2.66e-09,3.30e-06,pGL2_BRI1_bri1_T-bri1_T,ADS1,NA,NA,Up,Distal Columella_Up


In [29]:
sig_DE_fil
write.csv(sig_DE_fil, file = "./output/v4_pGL2-BRI1-bri1_T_v_bri1-T_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220112.csv")

gene,cluster_id,sc_131.cpm,sc_135.cpm,sc_133.cpm,sc_137.cpm,sc_131.frq,sc_135.frq,sc_133.frq,sc_137.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01470,Distal Columella,3270.00,3330.00,5940.00,5830.00,0.9860,0.9960,0.9890,0.9960,⋯,14.6,1.32e-04,7.12e-03,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,LEA14,NA,NA,Up,Distal Columella_Up
AT1G01480,Distal Columella,8.08,5.21,11.40,22.20,0.0599,0.0491,0.0852,0.2790,⋯,10.0,1.56e-03,4.62e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,ACS2,NA,NA,Up,Distal Columella_Up
AT1G01725,Distal Columella,4.44,4.01,26.70,26.90,0.0300,0.0377,0.1850,0.2880,⋯,45.8,1.34e-11,5.70e-09,7.77e-06,pGL2_BRI1_bri1_T-bri1_T,AT1G01725,NA,NA,Up,Distal Columella_Up
AT1G02390,Distal Columella,15.80,19.20,46.70,28.70,0.0978,0.1430,0.2310,0.2400,⋯,10.8,9.94e-04,3.35e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,GPAT2,NA,NA,Up,Distal Columella_Up
AT1G02860,Distal Columella,56.40,46.10,135.00,175.00,0.3010,0.2830,0.5070,0.7380,⋯,25.8,3.80e-07,5.58e-05,2.19e-01,pGL2_BRI1_bri1_T-bri1_T,BAH1,NA,NA,Up,Distal Columella_Up
AT1G03200,Distal Columella,4.24,6.02,11.30,62.00,0.0284,0.0491,0.0730,0.3990,⋯,38.0,7.10e-10,1.95e-07,4.11e-04,pGL2_BRI1_bri1_T-bri1_T,AT1G03200,NA,NA,Up,Distal Columella_Up
AT1G04040,Distal Columella,55.80,69.80,36.70,23.70,0.3440,0.4570,0.2310,0.2400,⋯,10.8,9.99e-04,3.35e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,AT1G04040,NA,NA,Down,Distal Columella_Down
AT1G05620,Distal Columella,54.50,51.70,31.50,20.80,0.3140,0.3360,0.2020,0.2270,⋯,10.3,1.30e-03,4.09e-02,1.00e+00,pGL2_BRI1_bri1_T-bri1_T,URH2,NA,NA,Down,Distal Columella_Down
AT1G06080,Distal Columella,2.42,1.20,22.90,10.20,0.0174,0.0113,0.1160,0.1160,⋯,47.5,5.70e-12,2.66e-09,3.30e-06,pGL2_BRI1_bri1_T-bri1_T,ADS1,NA,NA,Up,Distal Columella_Up


In [30]:
# add DE and up/dn to total list
sig_to_join <- sig_DE_fil %>%
mutate(clust_gene=paste(cluster_id, gene, sep="_")) %>%
select(clust_gene, up_dn_label, clust_up_dn)

In [31]:
# join all genes list to DE labels
all_bg <- mutate(all_bg, clust_gene=paste(cluster_id, gene, sep="_"))

all_bg <- left_join(all_bg, feature_names, by=c("gene"="AGI"))

all_bg$DE <- all_bg$clust_gene %in% sig_to_join$clust_gene


all_bg <- all_bg %>%
left_join(sig_to_join, by="clust_gene") %>%
arrange(all_bg, p_adj.loc)

write.csv(all_bg, file = "./output/v4_all_pGL2-BRI1-bri1_T_v_bri1-T_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220112.csv")